In [1]:
import os
import pandas as pd
from relm.histogram import Histogram
from relm.mechanisms import SmallDB
import numpy as np
from itertools import product
import scipy.sparse as sps


In [2]:
fp = 'examples/20200811_QLD_dummy_dataset_individual_v2.xlsx'
df = pd.read_excel(fp)
df.drop(["NOTF_ID", "LGA", "HHS"] + list(df.columns[12:]), axis=1, inplace=True)

hist = Histogram(df)

In [3]:
_cols = ["AGEGRP5", "SEX", "INDIG_STATUS"]
queries = []

# this creates the queries for:
# df.groupby(["AGEGRP5", "SEX", "INDIG_STATUS", col]).count()
# where col is in ["HOSPITALISED", "VENTILATED", "ICU", "DIED_OF_CONDITION"]
for col in ["HOSPITALISED", "VENTILATED", "ICU", "DIED_OF_CONDITION"]:
    cols = _cols + [col,]
    vals = product(*[list(hist.column_sets[hist.column_dict[c]]) for c in cols])
    queries.extend(dict(zip(cols, val)) for val in vals)

# this creates the queries for:
# df.groupby(["ONSET_DATE", "AGEGRP5", "INDIG_STATUS", "SEX"]).count()
cols = ["ONSET_DATE", "AGEGRP5", "INDIG_STATUS", "SEX"]
vals = product(*[list(hist.column_sets[hist.column_dict[c]]) for c in cols])
queries.extend(dict(zip(cols, val)) for val in vals)

# this creates the queries for:
# df.groupby(["ONSET_DATE", "POSTCODE"]).count()
cols = ["ONSET_DATE", "POSTCODE"]
vals = product(*[list(hist.column_sets[hist.column_dict[c]]) for c in cols])
queries.extend(dict(zip(cols, val)) for val in vals)

queries = sps.vstack([hist.get_query_vector(q) for q in queries])

In [4]:
smalldb = SmallDB(epsilon=4, data=hist.get_db(), alpha=0.1)

In [5]:
%time x = smalldb.release(queries)

CPU times: user 2min 1s, sys: 1.68 s, total: 2min 3s
Wall time: 2min 3s


In [6]:
x

array([0, 0, 0, ..., 1, 0, 0], dtype=uint64)

In [7]:
len(x)

28554240